In [75]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
import math

In [76]:
# Jupyter configuration
pd.set_option('display.max_rows', 500)

In [77]:
dataset = pd.read_csv('owid-covid-data.csv')

Separate the data into two datasets

In [78]:
moldova_dataset = dataset[dataset['location'] == 'Moldova']
moldova_dataset.to_csv('moldova-covid-data.csv')

dataset = moldova_dataset

In [79]:
# The shape of the dataset
dataset.shape

(604, 65)

### Variables description

| Variable                             | Description                                                                                                    |
|:-------------------------------------|:---------------------------------------------------------------------------------------------------------------|
| `icu_patients`                       | Number of COVID-19 patients in intensive care units (ICUs) on a given day                                      |
| `icu_patients_per_million`           | Number of COVID-19 patients in intensive care units (ICUs) on a given day per 1,000,000 people                 |
| `hosp_patients`                      | Number of COVID-19 patients in hospital on a given day                                                         |
| `hosp_patients_per_million`          | Number of COVID-19 patients in hospital on a given day per 1,000,000 people                                    |
| `weekly_icu_admissions`              | Number of COVID-19 patients newly admitted to intensive care units (ICUs) in a given week                      |
| `weekly_icu_admissions_per_million`  | Number of COVID-19 patients newly admitted to intensive care units (ICUs) in a given week per 1,000,000 people |
| `weekly_hosp_admissions`             | Number of COVID-19 patients newly admitted to hospitals in a given week                                        |
| `weekly_hosp_admissions_per_million` | Number of COVID-19 patients newly admitted to hospitals in a given week per 1,000,000 people                   |

### Policy responses
| Variable           | Description                                                                                                                                                                                                         |
|:-------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `stringency_index` | Government Response Stringency Index: composite measure based on 9 response indicators including school closures, workplace closures, and travel bans, rescaled to a value from 0 to 100 (100 = strictest response) |

### Reproduction rate
| Variable            | Description                                                                                                                                   |
|:--------------------|:----------------------------------------------------------------------------------------------------------------------------------------------|
| `reproduction_rate` | Real-time estimate of the effective reproduction rate (R) of COVID-19. See https://github.com/crondonm/TrackingR/tree/main/Estimates-Database |

### Tests & positivity
| Variable                          | Description                                                                                                                                                                                                                                                                                                          |
|:----------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `total_tests`                     | Total tests for COVID-19                                                                                                                                                                                                                                                                                             |
| `new_tests`                       | New tests for COVID-19 (only calculated for consecutive days)                                                                                                                                                                                                                                                        |
| `total_tests_per_thousand`        | Total tests for COVID-19 per 1,000 people                                                                                                                                                                                                                                                                            |
| `new_tests_per_thousand`          | New tests for COVID-19 per 1,000 people                                                                                                                                                                                                                                                                              |
| `new_tests_smoothed`              | New tests for COVID-19 (7-day smoothed). For countries that don't report testing data on a daily basis, we assume that testing changed equally on a daily basis over any periods in which no data was reported. This produces a complete series of daily figures, which is then averaged over a rolling 7-day window |
| `new_tests_smoothed_per_thousand` | New tests for COVID-19 (7-day smoothed) per 1,000 people                                                                                                                                                                                                                                                             |
| `positive_rate`                   | The share of COVID-19 tests that are positive, given as a rolling 7-day average (this is the inverse of tests_per_case)                                                                                                                                                                                              |
| `tests_per_case`                  | Tests conducted per new confirmed case of COVID-19, given as a rolling 7-day average (this is the inverse of positive_rate)                                                                                                                                                                                          |
| `tests_units`                     | Units used by the location to report its testing data 

Looking at these variables one can observe that for some variables there are pair variables that end wirh 'per_thousand' or 'per_million'. These variables generalize the information regarding the variable that is represented by tehm. We can exclude these variables because they have the same behaviour as the real measurements. For example, icu_patients in a way represents the same information as variable icu_patients_per_million and should have the same behaviour. The variables to be excluded are: 

* icu_patients_per_million
* hosp_patients_per_million
* weekly_icu_admissions_per_million
* weekly_hosp_admissions_per_million
* total_tests_per_thousand
* new_tests_per_thousand
* new_tests_smoothed
* new_tests_smoothed_per_thousand.

In [80]:
dataset.drop([
        'icu_patients_per_million',
        'hosp_patients_per_million', 
        'weekly_icu_admissions_per_million',
        'weekly_hosp_admissions_per_million',
        'total_tests_per_thousand',
        'new_tests_per_thousand',
        'new_tests_smoothed',
        'new_tests_smoothed_per_thousand'
    ],
    axis='columns',
    inplace=True
)

dataset.shape

(604, 57)

Select only a subset of variables to work with

In [81]:
dataset = dataset[[
    'location',
    'date',
    'icu_patients',
    'hosp_patients',
    'weekly_icu_admissions', 
    'weekly_hosp_admissions', 
    'stringency_index', 
    'reproduction_rate', 
    'total_tests', 
    'new_tests', 
    'positive_rate', 
    'tests_per_case', 
    'tests_units'
]]

dataset.shape

(604, 13)

In [82]:
# All available variables
dataset.columns.tolist()

['location',
 'date',
 'icu_patients',
 'hosp_patients',
 'weekly_icu_admissions',
 'weekly_hosp_admissions',
 'stringency_index',
 'reproduction_rate',
 'total_tests',
 'new_tests',
 'positive_rate',
 'tests_per_case',
 'tests_units']

In [83]:
# Fixing date column datatype
dataset['date'] = pd.to_datetime(dataset['date'])

/tmp/ipykernel_8340/1142007443.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['date'] = pd.to_datetime(dataset['date'])


In [84]:
# Variable types
dataset.dtypes

location                          object
date                      datetime64[ns]
icu_patients                     float64
hosp_patients                    float64
weekly_icu_admissions            float64
weekly_hosp_admissions           float64
stringency_index                 float64
reproduction_rate                float64
total_tests                      float64
new_tests                        float64
positive_rate                    float64
tests_per_case                   float64
tests_units                       object
dtype: object

#### There are two variables that are not numerical: date and tests_units. 

### Before dealing with missing values, let's take a look at the head and tail of the resulting dataset. 

In [85]:
dataset.head(10)

,location,date,icu_patients,hosp_patients,weekly_icu_admissions,weekly_hosp_admissions,stringency_index,reproduction_rate,total_tests,new_tests,positive_rate,tests_per_case,tests_units
75112,Moldova,2020-03-08,NaN,NaN,NaN,NaN,11.11,NaN,NaN,NaN,NaN,NaN,NaN
75113,Moldova,2020-03-09,NaN,NaN,NaN,NaN,11.11,NaN,NaN,NaN,NaN,NaN,NaN
75114,Moldova,2020-03-10,NaN,NaN,NaN,NaN,33.33,NaN,NaN,NaN,NaN,NaN,NaN
75115,Moldova,2020-03-11,NaN,NaN,NaN,NaN,51.85,NaN,NaN,NaN,NaN,NaN,NaN
75116,Moldova,2020-03-12,NaN,NaN,NaN,NaN,51.85,NaN,NaN,NaN,NaN,NaN,NaN
75117,Moldova,2020-03-13,NaN,NaN,NaN,NaN,51.85,NaN,NaN,NaN,NaN,NaN,NaN
75118,Moldova,2020-03-14,NaN,NaN,NaN,NaN,51.85,NaN,NaN,NaN,NaN,NaN,NaN
75119,Moldova,2020-03-15,NaN,NaN,NaN,NaN,51.85,NaN,NaN,NaN,NaN,NaN,NaN
75120,Moldova,2020-03-16,NaN,NaN,NaN,NaN,51.85,NaN,NaN,NaN,NaN,NaN,NaN
75121,Moldova,2020-03-17,NaN,NaN,NaN,NaN,54.63,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
dataset.tail(10)

,location,date,icu_patients,hosp_patients,weekly_icu_admissions,weekly_hosp_admissions,stringency_index,reproduction_rate,total_tests,new_tests,positive_rate,tests_per_case,tests_units
75706,Moldova,2021-10-23,NaN,NaN,NaN,NaN,25.93,1.07,NaN,NaN,0.169,5.9,tests performed
75707,Moldova,2021-10-24,NaN,NaN,NaN,NaN,25.93,1.04,NaN,NaN,0.184,5.4,tests performed
75708,Moldova,2021-10-25,NaN,NaN,NaN,NaN,25.93,1.01,1965179.0,NaN,0.205,4.9,tests performed
75709,Moldova,2021-10-26,NaN,NaN,NaN,NaN,25.93,0.97,NaN,NaN,NaN,NaN,NaN
75710,Moldova,2021-10-27,NaN,NaN,NaN,NaN,25.93,0.95,NaN,NaN,NaN,NaN,NaN
75711,Moldova,2021-10-28,NaN,NaN,NaN,NaN,25.93,0.94,NaN,NaN,NaN,NaN,NaN
75712,Moldova,2021-10-29,NaN,NaN,NaN,NaN,25.93,NaN,NaN,NaN,NaN,NaN,NaN
75713,Moldova,2021-10-30,NaN,NaN,NaN,NaN,25.93,NaN,NaN,NaN,NaN,NaN,NaN
75714,Moldova,2021-10-31,NaN,NaN,NaN,NaN,25.93,NaN,NaN,NaN,NaN,NaN,NaN
75715,Moldova,2021-11-01,NaN,NaN,NaN,NaN,25.93,NaN,NaN,NaN,NaN,NaN,NaN


All of the numerical variables are represented by `pandas` library as floats64. This is fine because there are a lot of missing values and soon we will replace them with mean or median, depending on the skewness, and these will not be integer numbers. 

Let's for now take a look at all unique values from the `tests_units` column.

This variable since its not numerical, it might have a weak relation to the other numerical variables and will not help much further when the prediction will be made.

In [87]:
dataset['tests_units'].value_counts()
dataset.drop('tests_units', axis='columns', inplace=True)

/home/ion/.local/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [88]:
stats_df = dataset.describe()
stats_df.loc['skewness'] = dataset.skew()
stats_df

/tmp/ipykernel_8340/3634021814.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  stats_df.loc['skewness'] = dataset.skew()


,icu_patients,hosp_patients,weekly_icu_admissions,weekly_hosp_admissions,stringency_index,reproduction_rate,total_tests,new_tests,positive_rate,tests_per_case
count,0.0,0.0,0.0,0.0,604.000000,584.000000,5.000000e+01,8.000000,155.000000,155.000000
mean,NaN,NaN,NaN,NaN,57.869123,1.067654,1.413658e+06,7443.375000,0.068194,40.078710
std,NaN,NaN,NaN,NaN,16.642418,0.223864,2.866976e+05,8092.509957,0.067061,33.011391
min,NaN,NaN,NaN,NaN,11.110000,0.570000,1.057234e+06,2593.000000,0.009000,3.700000
25%,NaN,NaN,NaN,NaN,48.150000,0.950000,1.135356e+06,4092.250000,0.015000,8.450000
50%,NaN,NaN,NaN,NaN,57.410000,1.090000,1.363792e+06,5063.500000,0.030000,33.300000
75%,NaN,NaN,NaN,NaN,67.590000,1.180000,1.636396e+06,6113.250000,0.118500,67.450000
max,NaN,NaN,NaN,NaN,87.040000,2.060000,1.965179e+06,27165.000000,0.271000,109.000000
skewness,NaN,NaN,NaN,NaN,-0.210294,0.500178,4.476820e-01,2.655809,1.092007,0.503356


In [89]:
dataset['location'].unique()

array(['Moldova'], dtype=object)